In [ ]:
from datetime import datetime
from tools import get_tools, get_tool_order
from latex_tools import tool_names, sc_plot
from ltlcross_runner import LtlcrossRunner
from IPython.display import display, display_markdown
import pandas as pd

If `rerun` is set to `False` the script uses the precomputed values. If set to `True`, all values are recomputed
(takes more time)

In [ ]:
rerun = True

Check if Buechic and Fribourg are installed.

In [ ]:
buechic = False
fribourg = False

# Refresh vars from store.
%store -r

In [ ]:
### Tools' setting ###
tools = get_tools(use_buechic=buechic, use_fribourg=fribourg)
tool_order = get_tool_order(use_buechic=buechic, use_fribourg=fribourg)


### Numbers to measure ###
cols = ['states','edges','transitions','nondet_states','exit_status','time','acc']

In [ ]:
runners = {}
for source in ('literature','random'):
    for t in ('det','cd','sd','nd'):
        name = '{}_{}'.format(source,t)
        runners[name] = \
            LtlcrossRunner(tools,\
                    res_filename='data/{}.csv'.format(name),\
                    formula_files=['formulae/{}.ltl'.format(name)],\
                    cols=cols)

In [ ]:
for name in runners.keys():
    print('{}: Working on {}'.format(datetime.now().strftime('[%d.%m.%Y %T]'),name))
    if rerun:
        runners[name].run_ltlcross(timeout='120')
    runners[name].parse_results()

In [ ]:
def get_counts(r):
    v = r.values.states.copy()
    return v.dropna().count()[0]

## Cummulative results

Gather the data into one big DataFrame

In [ ]:
data = pd.DataFrame()
for (name,r) in runners.items():
    row = pd.DataFrame(r.cummulative(),columns=[name]).transpose()
    data = data.append(row)
data

Get the number of formulas such that all tools finished within timeout for each category.

In [ ]:
counts = pd.DataFrame(index=pd.Index([],name='source'),columns=['.n'])
for (name,r) in runners.items():
    c = get_counts(r)
    counts = counts.append(pd.DataFrame({'.n':c},index=[name]))
counts

Converts the DataFrame to use MultiIndices for rows and columns

In [ ]:
from pandas.api.types import CategoricalDtype
automata_type = CategoricalDtype(categories=['det','cd','sd','nd'], ordered=True)
source_type = CategoricalDtype(categories=['random', 'literature', 'rand. (T/O)', 'lit. (T/O)'], ordered=True)

In [ ]:
def make_hierarchical(data):
    # Split index into origin & ltl2tgba's output type
    df = data.copy()
    df.index.name = 'origin'
    df.reset_index(inplace=True)
    df['type'] = df.origin.apply(lambda x: x.split('_')[1]).astype(automata_type)
    df['origin'] = df.origin.apply(lambda x: x.split('_')[0]).astype(source_type)
    df = df.set_index(['origin','type']).sort_index()
    # Split columns in tool & reductions
    df.columns = pd.MultiIndex.from_tuples([tuple(c.split('.')) for c in df.columns])
    return df

In [ ]:
tmp = make_hierarchical(data.join(counts)).sort_index(axis=1)
tmp

In [ ]:
ordered = tmp[['']+tool_order]

In [ ]:
ordered

In [ ]:
!mkdir figures

In [ ]:
with open("figures/ltl_table.fig", "w") as f:
    fixed_names = ordered.copy()
    fixed_names.columns.set_levels([[tool_names.get(item, item) for item in names] if i==0 else names
                                    for i, names in enumerate(fixed_names.columns.levels)], inplace=True)
    
    fixed = fixed_names.apply(pd.to_numeric, downcast="integer")
    
    fixed.to_latex(buf=f, escape=False, sparsify=True)

In [ ]:
for name, r in [(name, r) for name, r in runners.items() if '_nd' in name]:
    sc_plot(r, 'spot_ncsb.yes', 'pbs.yes', filename=f"figures/{name}_ncsb_pbs.fig", log="both")
    sc_plot(r, 'tgba_via_det.yes', 'pbs.yes', filename=f"figures/{name}_spot_pbs.fig", log="both")
    sc_plot(r, 'buechic.yes', 'pbs.yes', filename=f"figures/{name}_buechic_pbs.fig", log="both")
    sc_plot(r, 'fribourg.yes', 'pbs.yes', filename=f"figures/{name}_fribourg_pbs.fig", log="both")

## Timeouts

In [ ]:
TO_data = pd.DataFrame()
for (name,r) in runners.items():
    TO_data = TO_data.append(pd.DataFrame(r.get_error_count(err_type='timeout'),columns=[name]).transpose(),sort=True)
TO = make_hierarchical(TO_data).replace(0,pd.NaT).dropna(how='all').fillna(0)
TO

# Final results
We use the `display` function to propagate the results to notebooks that just run this one.

In [ ]:
display_markdown('''### Comparison of tools producing complements of given automata
Automata has been produced from formulae from literature and randomly generated.
There has been several timeouts.
''',raw=True)

In [ ]:
display(ordered)

In [ ]:
display_markdown('''### Timeouts
''',raw=True)

In [ ]:
display(TO)